## 第二题：分析高钾玻璃、铅钡玻璃的分类规律

In [ ]:
import  pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import sklearn.ensemble as ensemble 
import graphviz
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('C题.xlsx')
df = shuffle(df, random_state=42) 
df = df.reset_index(drop = True)
idx = 0
for i in df.loc[:,'类型']:
    if i == '高钾':
        df.loc[idx,'类型'] = 0
    idx += 1
df
idx = 0
for i in df.loc[:,'类型']:
    if i == '铅钡':
        df.loc[idx,'类型'] = 1
    idx += 1
df

In [ ]:
# 模型调参
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,3:],df.iloc[:,2],test_size=0.2)
param_grid = {
    'criterion':['entropy','gini'],
    'max_depth':[5, 6, 7, 8],
    'n_estimators':[11,13,15],  
    'max_features':[0.3,0.4,0.5],
    'min_samples_split':[4,8,12,16]  
}

rfc = ensemble.RandomForestClassifier()
rfc_cv = GridSearchCV(estimator=rfc, param_grid=param_grid,
                      scoring='roc_auc', cv=4)
rfc_cv.fit(X_train, y_train.astype('int'))
test_est = rfc_cv.predict(X_test)
print('精确矩阵')
print(metrics.classification_report(test_est, y_test.astype('int')))
print('AUC')
fpr_test, tpr_test, th_test = metrics.roc_curve(test_est, y_test.astype('int'))
print('AUC = %.4f' %metrics.auc(fpr_test, tpr_test))

In [ ]:
print('best_params,best_score:',rfc_cv.best_params_,rfc_cv.best_score_)

In [ ]:
# 找出重要特征
rfc = ensemble.RandomForestClassifier(criterion= 'entropy', max_depth= 5, max_features= 0.3, min_samples_split= 4, n_estimators= 11)
rfc.fit(X_train,y_train.astype('int'))
labels = df.columns[3:]
ipt = rfc.feature_importances_
indices = np.argsort(ipt)[::-1]
for i in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (i + 1, 30, labels[indices[i]], ipt[indices[i]]))

In [ ]:
# 可视化随机森林

estimator = rfc.estimators_[10]
data = tree.export_graphviz(estimator
                                ,feature_names=df.columns[3:] 
                                ,class_names=['高钾','铅钡'] 
                                ,filled=True
                                ,rounded=True
                                    ,out_file=None)
graph = graphviz.Source(data)
graph

## 第三题：鉴别表三所属类型

In [16]:
est = pd.read_excel('副本附件.xlsx')
est = est.fillna(0)
X_test = est.iloc[:,2:]
result = rfc.predict(X_test)
result

array([0, 1, 1, 1, 1, 0, 0, 1])